#ResNext + LSTM 모델을 활용

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torchvision.transforms.functional import to_tensor
from PIL import Image  # PIL의 Image 모듈을 import합니다.

import glob
import torch
import torchvision
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=26bb28769174d57fd3347a340996d9eb382cd6e9387990efde1cb2cdb40372f8
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import face_recognition

# Function to check if an image file is corrupted or not -> 필요없어졌는 지 check
def is_corrupted_image(image_path):
    try:
        _ = cv2.imread(image_path)
        return False
    except Exception as e:
        print(f"Corrupted image: {image_path}")
        return True

# Function to validate a list of image files -> 시간이 오래 걸려서 일단 생략.
def validate_images(image_paths, train_transforms):
    validated_images = []
    for image_path in image_paths:
        try:
            image = cv2.imread(image_path)
            if image is None:
                print(f"Invalid image: {image_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(image)
            pil_image = train_transforms(pil_image)
            validated_images.append(pil_image)
        except Exception as e:
            print(f"Error processing image: {image_path}")
            continue
    return validated_images

def extract_file_names(data_list):
    desired_parts_list = []
    for file_path in data_list:
        parts = file_path.split('/')
        desired_parts = parts[5:] #/content/drive/MyDrive/datasets/valid/real/dsaass.jpg
        new_file_path = '/'.join(desired_parts)
        desired_parts_list.append(new_file_path)
    return desired_parts_list

# Constants and configurations
#im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define transformations
train_transforms = transforms.Compose([
    #transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std) #표준화 이용. 이미지 정규화 하는 부분에서는 Normalization (0-1) 사이의 값, 혹은 Min-Max (-1,1) 사이의 값들을 이룬다.
])

# Paths to image directories, #list형태 사용
train_image_dirs = [
    '/content/drive/MyDrive/datasets/valid/real/',
    '/content/drive/MyDrive/datasets/valid/fake/'
]

test_image_dirs = [
    '/content/drive/MyDrive/datasets/test/real/',
    '/content/drive/MyDrive/datasets/test/fake/'
]

# Get list of all train image files
train_image_files = []  #image file들을 저장.(훈련 데이터만 저장을 한다)
for dir_path in train_image_dirs:
    train_image_files += glob.glob(os.path.join(dir_path, '*.jpg'))  # Assuming images are JPEG format

random.shuffle(train_image_files)
random.shuffle(train_image_files)

#Get list of all test image files
test_image_files = []  #image file들을 저장.(테스트 데이터만 저장을 한다)
for dir_path in test_image_dirs:
    test_image_files += glob.glob(os.path.join(dir_path, '*.jpg'))  # Assuming images are JPEG format

random.shuffle(test_image_files)
random.shuffle(test_image_files)

print("Total number of images:", len(train_image_files)) #train_image_files 2x10000
print("Total number of images:", len(test_image_files)) #test_image_files은 2x10000

# Validate images
#validated_images = validate_images(train_image_files, train_transforms) 시간이 너무 오래걸린다.

#print("Total number of validated images:", len(validated_images))

In [ ]:
# load the image name and labels from csv
class image_dataset(Dataset):
    def __init__(self,image_names,labels,transform = None):
        self.image_names = image_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = self.image_names[idx]
        label_info = self.labels.loc[self.labels['path'].values == image_path]

        if not label_info.empty:
            label_str = label_info['label_str'].values[0]
            if label_str == 'fake':
                label = 0
            elif label_str == 'real':
                label = 1
        else:
            label = -1  # 라벨 정보가 없는 경우 처리 (원하는 방식으로 수정 필요)

       # 이미지를 텐서로 변환
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)

        return image, label

#plot the image
def im_plot(tensor):
    image_np = np.array(tensor)  # PIL 이미지를 넘파이 배열로 변환
    plt.imshow(image_np)
    plt.show()


In [ ]:
#count the number of fake and real videos, Because of random shuffle, there is difference.
def number_of_real_and_fake_videos(data_list, lab):
    if data_list == train_images or data_list == valid_images:
        real = 0
        fake = 0
        data_list = extract_file_names(data_list)
        for names in data_list:
          label_info = lab.loc[lab["path"].values == names]
           #해당하는 열의 정보를 label_info에 저장을 한다.
          if not label_info.empty:
            label_str = label_info['label_str'].values[0]  # 라벨 정보가 존재한다면 해당 이미지의 라벨 문자열 가져오기
            if label_str == 'real':
                real += 1
            elif label_str == 'fake':
                fake += 1
    else:
        real = 0
        fake = 0
        data_list = extract_file_names(data_list)
        for names in data_list:
          label_info = lab.loc[lab["path"].values == names]
          if not label_info.empty:
            label_str = label_info['label_str'].values[0]  # 라벨 정보가 존재한다면 해당 이미지의 라벨 문자열 가져오기
            if label_str == 'real':
                real += 1
            elif label_str == 'fake':
                fake += 1
    return real, fake

In [ ]:
def custom_collate_fn(batch):
    # 배치를 텐서로 변환
    images = [to_tensor(item[0]) for item in batch]
    labels = [torch.tensor(item[1]) for item in batch]  # 라벨도 텐서로 변환

    # 이미지 텐서로 변환
    images = torch.stack(images)

    return images, labels

In [ ]:
# load the labels and video in data loader
import random
import pandas as pd
from sklearn.model_selection import train_test_split

header_list = ["number","original_path","id","label","label_str","path"] #우리의 csv파일에 맞게 label list형태를 변환.
label_train = pd.read_csv('/content/drive/MyDrive/datasets/labels/train.csv', names= header_list)
label_test = pd.read_csv('/content/drive/MyDrive/datasets/labels/test.csv', names = header_list)

#csv필요한 부분만 읽기.
label_train = label_train.drop(columns=["number", "original_path", "id"])
label_test = label_test.drop(columns=["number", "original_path", "id"])
#print(labels)
#train image, validation, test image 만드는 코드 (image_file에는 2만장 real, fake가 섞여있는 객체이고 2만장을 8:2로 각각 train : 16000, validation: 4000장)
#test image도 4천장 정도 만들어야 한다.

train_images = train_image_files[:int(0.8*len(train_image_files))] #처음부터 80%까지의 data를 저장
valid_images = train_image_files[int(0.8*len(train_image_files)):] #20% data는 validation image로 저장.
test_images = test_image_files[:int(0.2*len(test_image_files))] #20% 즉 4000장 정도 -> 나중에 train_test_split을 통해서 real과 fake의 비율을 맞추는 개선 활동 필요.

print("train : " , len(train_images))
print("validation : ", len(valid_images))
print("test : " , len(test_images))

#train_images,valid_images = train_test_split(image_files,test_size = 0.2)
#print(train_videos)
#real, fake = number_of_real_and_fake_videos(train_images,label_train)
#print("TRAIN: ", "Real:",real," Fake:",fake)

#real, fake = number_of_real_and_fake_videos(valid_images,label_train)
#print("VALIDATION: ", "Real:",real," Fake:",fake)

#real, fake = number_of_real_and_fake_videos(test_images,label_test)
#print("TEST: ", "Real:",real," Fake:",fake)

#전부 절반 정도 real, fake이미지를 가지고 있다.

im_size = 128 #256/2
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])

#train_dataset (return image와 label정보)
train_data = image_dataset(train_images,label_train,transform = None)
val_data = image_dataset(valid_images,label_train,transform = None)

#train dataloader
train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2, collate_fn=custom_collate_fn) #num_worker는 subprocess이다.
valid_loader = DataLoader(val_data, batch_size=4, shuffle=True, num_workers=2, collate_fn=custom_collate_fn)
image,label = train_data[0]
#im_plot(image[0,:,:,:])

In [ ]:
#Model with feature visualization
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))

In [ ]:
model = Model(2).cuda()
a,b = model(torch.from_numpy(np.empty((1,20,3,112,112))).type(torch.cuda.FloatTensor))

In [ ]:
import torch
from torch.autograd import Variable
import time
import os
import sys
import os
def train_epoch(epoch, num_epochs, data_loader, model, criterion, optimizer):
    model.train()
    losses = AverageMeter()
    accuracies = AverageMeter()
    t = []
    for i, (inputs, targets) in enumerate(data_loader):
        if torch.cuda.is_available():
            targets = targets.type(torch.cuda.LongTensor)
            inputs = inputs.cuda()
        _,outputs = model(inputs)
        loss  = criterion(outputs,targets.type(torch.cuda.LongTensor))
        acc = calculate_accuracy(outputs, targets.type(torch.cuda.LongTensor))
        losses.update(loss.item(), inputs.size(0))
        accuracies.update(acc, inputs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d / %d] [Loss: %f, Acc: %.2f%%]"
                % (
                    epoch,
                    num_epochs,
                    i,
                    len(data_loader),
                    losses.avg,
                    accuracies.avg))
    torch.save(model.state_dict(),'/content/checkpoint.pt')
    return losses.avg,accuracies.avg
def test(epoch,model, data_loader ,criterion):
    print('Testing')
    model.eval()
    losses = AverageMeter()
    accuracies = AverageMeter()
    pred = []
    true = []
    count = 0
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            if torch.cuda.is_available():
                targets = targets.cuda().type(torch.cuda.FloatTensor)
                inputs = inputs.cuda()
            _,outputs = model(inputs)
            loss = torch.mean(criterion(outputs, targets.type(torch.cuda.LongTensor)))
            acc = calculate_accuracy(outputs,targets.type(torch.cuda.LongTensor))
            _,p = torch.max(outputs,1)
            true += (targets.type(torch.cuda.LongTensor)).detach().cpu().numpy().reshape(len(targets)).tolist()
            pred += p.detach().cpu().numpy().reshape(len(p)).tolist()
            losses.update(loss.item(), inputs.size(0))
            accuracies.update(acc, inputs.size(0))
            sys.stdout.write(
                    "\r[Batch %d / %d]  [Loss: %f, Acc: %.2f%%]"
                    % (
                        i,
                        len(data_loader),
                        losses.avg,
                        accuracies.avg
                        )
                    )
        print('\nAccuracy {}'.format(accuracies.avg))
    return true,pred,losses.avg,accuracies.avg
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
def calculate_accuracy(outputs, targets):
    batch_size = targets.size(0)

    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1))
    n_correct_elems = correct.float().sum().item()
    return 100* n_correct_elems / batch_size

In [ ]:
import seaborn as sn
#Output confusion matrix
def print_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    print('True positive = ', cm[0][0])
    print('False positive = ', cm[0][1])
    print('False negative = ', cm[1][0])
    print('True negative = ', cm[1][1])
    print('\n')
    df_cm = pd.DataFrame(cm, range(2), range(2))
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size
    plt.ylabel('Actual label', size = 20)
    plt.xlabel('Predicted label', size = 20)
    plt.xticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.yticks(np.arange(2), ['Fake', 'Real'], size = 16)
    plt.ylim([2, 0])
    plt.show()
    calculated_acc = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+ cm[1][1])
    print("Calculated Accuracy",calculated_acc*100)

In [ ]:
def plot_loss(train_loss_avg,test_loss_avg,num_epochs):
  loss_train = train_loss_avg
  loss_val = test_loss_avg
  print(num_epochs)
  epochs = range(1,num_epochs+1)
  plt.plot(epochs, loss_train, 'g', label='Training loss')
  plt.plot(epochs, loss_val, 'b', label='validation loss')
  plt.title('Training and Validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
def plot_accuracy(train_accuracy,test_accuracy,num_epochs):
  loss_train = train_accuracy
  loss_val = test_accuracy
  epochs = range(1,num_epochs+1)
  plt.plot(epochs, loss_train, 'g', label='Training accuracy')
  plt.plot(epochs, loss_val, 'b', label='validation accuracy')
  plt.title('Training and Validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
#learning rate
lr = 1e-5#0.001
#number of epochs
num_epochs = 20

optimizer = torch.optim.Adam(model.parameters(), lr= lr,weight_decay = 1e-5)

#class_weights = torch.from_numpy(np.asarray([1,15])).type(torch.FloatTensor).cuda()
#criterion = nn.CrossEntropyLoss(weight = class_weights).cuda()
criterion = nn.CrossEntropyLoss().cuda()
train_loss_avg =[]
train_accuracy = []
test_loss_avg = []
test_accuracy = []
for epoch in range(1,num_epochs+1):
    l, acc = train_epoch(epoch,num_epochs,train_loader,model,criterion,optimizer)
    train_loss_avg.append(l)
    train_accuracy.append(acc)
    true,pred,tl,t_acc = test(epoch,model,valid_loader,criterion)
    test_loss_avg.append(tl)
    test_accuracy.append(t_acc)
plot_loss(train_loss_avg,test_loss_avg,len(train_loss_avg))
plot_accuracy(train_accuracy,test_accuracy,len(train_accuracy))
print(confusion_matrix(true,pred))
print_confusion_matrix(true,pred)